## Prep

In [1]:
import numpy as np
import pandas as pd
import warnings
from scipy.linalg import solve_continuous_lyapunov
from sklearn import linear_model
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)


# -------------- commutation matrix --------------
def comm_mat(m, n):
    w = np.arange(m * n).reshape((m, n), order="F").T.ravel(order="F")
    return np.eye(m * n)[w, :]

# -------------- Gram matrix --------------
def gram(A: np.ndarray) -> np.ndarray:
    return np.matmul(np.transpose(A), A)

# -------------- matrix A(Sigma) --------------
def create_A_Sigma(cov: np.ndarray, p: int) -> np.ndarray:
    identity = np.eye(p)
    return np.kron(cov, identity) + np.matmul(np.kron(identity, cov), comm_mat(p, p))

def create_A_Sigma_hard(cov: np.ndarray, p: int, index: int) -> np.ndarray:
    identity = np.eye(p)
    identity[index, index] = 0
    if p == index:
        cov = np.concatenate((cov, np.zeros((1, p-1))), axis=0)
        cov = np.concatenate((cov, np.zeros((p, 1))), axis=1)
    else:
        cov = np.insert(cov, index, 0, axis=0)
        cov = np.insert(cov, index, 0, axis=1)
    return np.kron(cov, identity) + np.matmul(np.kron(identity, cov), comm_mat(p, p))



# -------------- matrix A (intervention) --------------
def create_A_shift(cov: np.ndarray, mean_int: np.ndarray, p: int) -> np.ndarray:
    identity = np.eye(p)
    A_1 = create_A_Sigma(cov, p)
    temp = np.kron(mean_int, identity)
    return np.vstack((A_1, temp))

def create_A_hard(cov: np.ndarray, cov_int:np.ndarray, mean_int: np.ndarray, p: int, index: int) -> np.ndarray:
    identity = np.eye(p)
    identity[index, index] = 0
    mean_int = np.insert(arr=mean_int, obj=index, values=hard_param, axis=0)
    A_1 = create_A_Sigma(cov, p)
    A_2 = create_A_Sigma_hard(cov_int, p, index)
    temp = np.kron(mean_int, identity)
    return np.vstack((A_1, A_2, temp))

# -------------- irrepresentability condition --------------
def irrep_cond(A: np.ndarray, support: np.ndarray, supp_compl: np.ndarray) -> float:
    gram_matrix = gram(A=A)
    try:
        matrix_1 = gram_matrix[supp_compl[:, None], support] # uses broadcasting
        matrix_2 = np.linalg.inv(gram_matrix[np.ix_(support, support)])
        return np.linalg.norm(np.matmul(matrix_1, matrix_2), ord=np.inf)
    except np.linalg.LinAlgError:
        return -1

# -------------- weak irrepresentability condition --------------
def weak_irrep_cond(M: np.ndarray, A: np.ndarray, support: np.ndarray, supp_compl: np.ndarray) -> float:
    gram_matrix = gram(A=A)
    try:
        matrix_1 = gram_matrix[supp_compl[:, None], support] # uses broadcasting
        matrix_2 = np.linalg.inv(gram_matrix[np.ix_(support, support)])
        sign_vec = np.sign(M.flatten(order="F"))[support]
        return np.linalg.norm(np.matmul(matrix_1, matrix_2) @ sign_vec, ord=np.inf)
    except np.linalg.LinAlgError:
        return -1




num_exp = 100000 # number of true matrices per case
volatility_param = 2
shift_param = 2
hard_param = 2
reg_param = 0.001

## 1 edge

<img src="Graphics/graph1.png" alt="Drawing" style="width: 100px;"/>

In [2]:
p = 2
seed = 0
index = 0

index_zero = np.array([[0, 1]])
support = np.array([0, 2, 3])
supp_compl = np.array([1])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_1edge = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_1edge_hard = ex_1edge.copy()
ex_1edge_shift = ex_1edge.copy()

for row in ex_1edge.itertuples():
    row = row.Index
    true_M = ex_1edge.iloc[row, 0]

    intervention_index = ex_1edge.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_1edge.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_1edge.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_1edge_shift.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_1edge_shift.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_1edge_hard.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_1edge_hard.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_1edge = ex_1edge[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_1edge_hard = ex_1edge_hard[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_1edge_shift = ex_1edge_shift[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_1edge.to_pickle("Experiments/irrep_DAG_1edge_1.pkl")
ex_1edge_hard.to_pickle("Experiments/irrep_DAG_1edge_hard.pkl")
ex_1edge_shift.to_pickle("Experiments/irrep_DAG_1edge_shift.pkl")

## 2 edges

### 3 nodes - Ex. 1

<img src="Graphics/graph2.png" alt="Drawing" style="width: 100px;"/>

In [3]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [1, 2], [2, 0]])
support = np.array([0, 3, 4, 7, 8])
supp_compl = np.array([1, 2, 5, 6])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_2edge_1 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_2edge_hard_1 = ex_2edge_1.copy()
ex_2edge_shift_1 = ex_2edge_1.copy()

for row in ex_2edge_1.itertuples():
    row = row.Index
    true_M = ex_2edge_1.iloc[row, 0]

    intervention_index = ex_2edge_1.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_2edge_1.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_2edge_1.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_2edge_shift_1.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_2edge_shift_1.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_2edge_hard_1.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_2edge_hard_1.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_2edge_1 = ex_2edge_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_2edge_hard_1 = ex_2edge_hard_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_2edge_shift_1 = ex_2edge_shift_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_2edge_1.to_pickle("Experiments/irrep_DAG_2edge_1.pkl")
ex_2edge_hard_1.to_pickle("Experiments/irrep_DAG_2edge_hard_1.pkl")
ex_2edge_shift_1.to_pickle("Experiments/irrep_DAG_2edge_shift_1.pkl")

### 3 nodes - Ex. 2
<img src="Graphics/graph3.png" alt="Drawing" style="width: 100px;"/>

In [4]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [2, 0], [2, 1]])
support = np.array([0, 3, 4, 5, 8])
supp_compl = np.array([1, 2, 6, 7])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_2edge_2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_2edge_hard_2 = ex_2edge_2.copy()
ex_2edge_shift_2 = ex_2edge_2.copy()

for row in ex_2edge_2.itertuples():
    row = row.Index
    true_M = ex_2edge_2.iloc[row, 0]

    intervention_index = ex_2edge_2.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_2edge_2.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_2edge_2.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_2edge_shift_2.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_2edge_shift_2.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_2edge_hard_2.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_2edge_hard_2.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_2edge_2 = ex_2edge_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_2edge_hard_2 = ex_2edge_hard_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_2edge_shift_2 = ex_2edge_shift_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_2edge_2.to_pickle("Experiments/irrep_DAG_2edge_2.pkl")
ex_2edge_hard_2.to_pickle("Experiments/irrep_DAG_2edge_hard_2.pkl")
ex_2edge_shift_2.to_pickle("Experiments/irrep_DAG_2edge_shift_2.pkl")

### 3 nodes - Ex. 3
<img src="Graphics/graph4.png" alt="Drawing" style="width: 100px;"/>

In [5]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 2], [1, 0], [1, 2], [2, 0]])
support = np.array([0, 1, 4, 7, 8])
supp_compl = np.array([2, 3, 5, 6])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_2edge_3 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_2edge_hard_3 = ex_2edge_3.copy()
ex_2edge_shift_3 = ex_2edge_3.copy()

for row in ex_2edge_3.itertuples():
    row = row.Index
    true_M = ex_2edge_3.iloc[row, 0]

    intervention_index = ex_2edge_3.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_2edge_3.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_2edge_3.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_2edge_shift_3.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_2edge_shift_3.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_2edge_hard_3.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_2edge_hard_3.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_2edge_3 = ex_2edge_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_2edge_hard_3 = ex_2edge_hard_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_2edge_shift_3 = ex_2edge_shift_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_2edge_3.to_pickle("Experiments/irrep_DAG_2edge_3.pkl")
ex_2edge_hard_3.to_pickle("Experiments/irrep_DAG_2edge_hard_3.pkl")
ex_2edge_shift_3.to_pickle("Experiments/irrep_DAG_2edge_shift_3.pkl")

## 3 edges

### 3 nodes - Ex. 1

<img src="Graphics/graph5.png" alt="Drawing" style="width: 100px;"/>

In [6]:
p = 3
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [1, 2]])
support = np.array([0, 3, 4, 6, 7, 8])
supp_compl = np.array([1, 2, 5])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_3edge_1 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_3edge_hard_1 = ex_3edge_1.copy()
ex_3edge_shift_1 = ex_3edge_1.copy()

for row in ex_3edge_1.itertuples():
    row = row.Index
    true_M = ex_3edge_1.iloc[row, 0]

    intervention_index = ex_3edge_1.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_3edge_1.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_3edge_1.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_3edge_shift_1.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_3edge_shift_1.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_3edge_hard_1.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_3edge_hard_1.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_3edge_1 = ex_3edge_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_hard_1 = ex_3edge_hard_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_shift_1 = ex_3edge_shift_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_1.to_pickle("Experiments/irrep_DAG_3edge_1.pkl")
ex_3edge_hard_1.to_pickle("Experiments/irrep_DAG_3edge_hard_1.pkl")
ex_3edge_shift_1.to_pickle("Experiments/irrep_DAG_3edge_shift_1.pkl")

### 4 nodes - Ex. 2

<img src="Graphics/graph6.png" alt="Drawing" style="width: 100px;"/>

In [7]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 0], [2, 3], [3, 0], [3, 1]])
support = np.array([0, 4, 5, 9, 10, 14, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 8, 11, 12, 13])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_3edge_2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_3edge_hard_2 = ex_3edge_2.copy()
ex_3edge_shift_2 = ex_3edge_2.copy()

for row in ex_3edge_2.itertuples():
    row = row.Index
    true_M = ex_3edge_2.iloc[row, 0]

    intervention_index = ex_3edge_2.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_3edge_2.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_3edge_2.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_3edge_shift_2.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_3edge_shift_2.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_3edge_hard_2.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_3edge_hard_2.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_3edge_2 = ex_3edge_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_hard_2 = ex_3edge_hard_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_shift_2 = ex_3edge_shift_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_2.to_pickle("Experiments/irrep_DAG_3edge_2.pkl")
ex_3edge_hard_2.to_pickle("Experiments/irrep_DAG_3edge_hard_2.pkl")
ex_3edge_shift_2.to_pickle("Experiments/irrep_DAG_3edge_shift_2.pkl")

### 4 nodes - Ex. 3
<img src="Graphics/graph7.png" alt="Drawing" style="width: 100px;"/>

In [8]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [0, 3], [1, 0], [1, 2], [1, 3], [2, 0], [2, 3], [3, 0], [3, 1]])
support = np.array([0, 1, 5, 9, 10, 14, 15])
supp_compl = np.array([2, 3, 4, 6, 7, 8, 11, 12, 13])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_3edge_3 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_3edge_hard_3 = ex_3edge_3.copy()
ex_3edge_shift_3 = ex_3edge_3.copy()

for row in ex_3edge_3.itertuples():
    row = row.Index
    true_M = ex_3edge_3.iloc[row, 0]

    intervention_index = ex_3edge_3.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_3edge_3.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_3edge_3.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_3edge_shift_3.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_3edge_shift_3.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_3edge_hard_3.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_3edge_hard_3.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_3edge_3 = ex_3edge_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_hard_3 = ex_3edge_hard_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_shift_3 = ex_3edge_shift_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_3.to_pickle("Experiments/irrep_DAG_3edge_3.pkl")
ex_3edge_hard_3.to_pickle("Experiments/irrep_DAG_3edge_hard_3.pkl")
ex_3edge_shift_3.to_pickle("Experiments/irrep_DAG_3edge_shift_3.pkl")

### 4 nodes - Ex. 4
<img src="Graphics/graph8.png" alt="Drawing" style="width: 100px;"/>

In [9]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 0], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 9, 10, 11, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 8, 11, 12, 13])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_3edge_4 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_3edge_hard_4 = ex_3edge_4.copy()
ex_3edge_shift_4 = ex_3edge_4.copy()

for row in ex_3edge_4.itertuples():
    row = row.Index
    true_M = ex_3edge_4.iloc[row, 0]

    intervention_index = ex_3edge_4.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_3edge_4.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_3edge_4.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_3edge_shift_4.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_3edge_shift_4.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_3edge_hard_4.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_3edge_hard_4.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_3edge_4 = ex_3edge_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_hard_4 = ex_3edge_hard_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_shift_4 = ex_3edge_shift_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_4.to_pickle("Experiments/irrep_DAG_3edge_4.pkl")
ex_3edge_hard_4.to_pickle("Experiments/irrep_DAG_3edge_hard_4.pkl")
ex_3edge_shift_4.to_pickle("Experiments/irrep_DAG_3edge_shift_4.pkl")

### 4 nodes - Ex. 5
<img src="Graphics/graph9.png" alt="Drawing" style="width: 100px;"/>

In [10]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [0, 3], [1, 0], [1, 2], [1, 3], [2, 0], [2, 3], [3, 0], [3, 1]])
support = np.array([0, 1, 5, 9, 10, 14, 15])
supp_compl = np.array([2, 3, 4, 6, 7, 8, 11, 12, 13])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_3edge_5 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_3edge_hard_5 = ex_3edge_5.copy()
ex_3edge_shift_5 = ex_3edge_5.copy()

for row in ex_3edge_5.itertuples():
    row = row.Index
    true_M = ex_3edge_5.iloc[row, 0]

    intervention_index = ex_3edge_5.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_3edge_5.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_3edge_5.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_3edge_shift_5.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_3edge_shift_5.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_3edge_hard_5.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_3edge_hard_5.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_3edge_5 = ex_3edge_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_hard_5 = ex_3edge_hard_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_shift_5 = ex_3edge_shift_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_5.to_pickle("Experiments/irrep_DAG_3edge_5.pkl")
ex_3edge_hard_5.to_pickle("Experiments/irrep_DAG_3edge_hard_5.pkl")
ex_3edge_shift_5.to_pickle("Experiments/irrep_DAG_3edge_shift_5.pkl")

### 4 nodes - Ex. 6
<img src="Graphics/graph10.png" alt="Drawing" style="width: 100px;"/>

In [11]:
p = 4
seed = 0
index = 0

index_zero = np.array([[1, 0], [1, 2], [1, 3], [2, 0], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 2, 3, 5, 10, 15])
supp_compl = np.array([4, 6, 7, 8, 9, 11, 12, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_3edge_6 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_3edge_hard_6 = ex_3edge_6.copy()
ex_3edge_shift_6 = ex_3edge_6.copy()

for row in ex_3edge_6.itertuples():
    row = row.Index
    true_M = ex_3edge_6.iloc[row, 0]

    intervention_index = ex_3edge_6.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_3edge_6.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_3edge_6.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_3edge_shift_6.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_3edge_shift_6.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_3edge_hard_6.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_3edge_hard_6.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_3edge_6 = ex_3edge_6[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_hard_6 = ex_3edge_hard_6[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_shift_6 = ex_3edge_shift_6[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_6.to_pickle("Experiments/irrep_DAG_3edge_6.pkl")
ex_3edge_hard_6.to_pickle("Experiments/irrep_DAG_3edge_hard_6.pkl")
ex_3edge_shift_6.to_pickle("Experiments/irrep_DAG_3edge_shift_6.pkl")

### 4 nodes - Ex. 7
<img src="Graphics/graph11.png" alt="Drawing" style="width: 100px;"/>

In [12]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [1, 2], [1, 3], [2, 0], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 2, 3, 4, 5, 10, 15])
supp_compl = np.array([1, 6, 7, 8, 9, 11, 12, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_3edge_7 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_3edge_hard_7 = ex_3edge_7.copy()
ex_3edge_shift_7 = ex_3edge_7.copy()

for row in ex_3edge_7.itertuples():
    row = row.Index
    true_M = ex_3edge_7.iloc[row, 0]

    intervention_index = ex_3edge_7.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_3edge_7.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_3edge_7.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_3edge_shift_7.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_3edge_shift_7.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_3edge_hard_7.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_3edge_hard_7.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_3edge_7 = ex_3edge_7[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_hard_7 = ex_3edge_hard_7[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_shift_7 = ex_3edge_shift_7[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_7.to_pickle("Experiments/irrep_DAG_3edge_7.pkl")
ex_3edge_hard_7.to_pickle("Experiments/irrep_DAG_3edge_hard_7.pkl")
ex_3edge_shift_7.to_pickle("Experiments/irrep_DAG_3edge_shift_7.pkl")

### 4 nodes - Ex. 8
<img src="Graphics/graph12.png" alt="Drawing" style="width: 100px;"/>

In [13]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 1], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 3, 4, 5, 8, 10, 15])
supp_compl = np.array([1, 2, 6, 7, 9, 11, 12, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_3edge_8 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_3edge_hard_8 = ex_3edge_8.copy()
ex_3edge_shift_8 = ex_3edge_8.copy()

for row in ex_3edge_8.itertuples():
    row = row.Index
    true_M = ex_3edge_8.iloc[row, 0]

    intervention_index = ex_3edge_8.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_3edge_8.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_3edge_8.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_3edge_shift_8.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_3edge_shift_8.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_3edge_hard_8.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_3edge_hard_8.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_3edge_8 = ex_3edge_8[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_hard_8 = ex_3edge_hard_8[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_shift_8 = ex_3edge_shift_8[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_8.to_pickle("Experiments/irrep_DAG_3edge_8.pkl")
ex_3edge_hard_8.to_pickle("Experiments/irrep_DAG_3edge_hard_8.pkl")
ex_3edge_shift_8.to_pickle("Experiments/irrep_DAG_3edge_shift_8.pkl")

### 4 nodes - Ex. 9
<img src="Graphics/graph13.png" alt="Drawing" style="width: 100px;"/>

In [14]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 1], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 8, 10, 12, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 9, 11, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_3edge_9 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_3edge_hard_9 = ex_3edge_9.copy()
ex_3edge_shift_9 = ex_3edge_9.copy()

for row in ex_3edge_9.itertuples():
    row = row.Index
    true_M = ex_3edge_9.iloc[row, 0]

    intervention_index = ex_3edge_9.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_3edge_9.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_3edge_9.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_3edge_shift_9.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_3edge_shift_9.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_3edge_hard_9.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_3edge_hard_9.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_3edge_9 = ex_3edge_9[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_hard_9 = ex_3edge_hard_9[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_shift_9 = ex_3edge_shift_9[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_3edge_9.to_pickle("Experiments/irrep_DAG_3edge_9.pkl")
ex_3edge_hard_9.to_pickle("Experiments/irrep_DAG_3edge_hard_9.pkl")
ex_3edge_shift_9.to_pickle("Experiments/irrep_DAG_3edge_shift_9.pkl")

## 4 edges

### Ex. 1
<img src="Graphics/graph14.png" alt="Drawing" style="width: 100px;"/>

In [15]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 0], [2, 3], [3, 1]])
support = np.array([0, 4, 5, 9, 10, 12, 14, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 8, 11, 13])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_4edge_1 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_4edge_hard_1 = ex_4edge_1.copy()
ex_4edge_shift_1 = ex_4edge_1.copy()

for row in ex_4edge_1.itertuples():
    row = row.Index
    true_M = ex_4edge_1.iloc[row, 0]

    intervention_index = ex_4edge_1.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_4edge_1.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_4edge_1.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_4edge_shift_1.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_4edge_shift_1.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_4edge_hard_1.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_4edge_hard_1.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_4edge_1 = ex_4edge_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_hard_1 = ex_4edge_hard_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_shift_1 = ex_4edge_shift_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_1.to_pickle("Experiments/irrep_DAG_4edge_1.pkl")
ex_4edge_hard_1.to_pickle("Experiments/irrep_DAG_4edge_hard_1.pkl")
ex_4edge_shift_1.to_pickle("Experiments/irrep_DAG_4edge_shift_1.pkl")

### Ex. 2
<img src="Graphics/graph15.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 3], [2, 0], [2, 1], [2, 3], [3, 1]])
support = np.array([0, 4, 5, 6, 10, 12, 14, 15])
supp_compl = np.array([1, 2, 3, 7, 8, 9, 11, 13])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_4edge_2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_4edge_hard_2 = ex_4edge_2.copy()
ex_4edge_shift_2 = ex_4edge_2.copy()

for row in ex_4edge_2.itertuples():
    row = row.Index
    true_M = ex_4edge_2.iloc[row, 0]

    intervention_index = ex_4edge_2.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_4edge_2.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_4edge_2.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_4edge_shift_2.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_4edge_shift_2.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_4edge_hard_2.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_4edge_hard_2.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_4edge_2 = ex_4edge_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_hard_2 = ex_4edge_hard_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_shift_2 = ex_4edge_shift_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_2.to_pickle("Experiments/irrep_DAG_4edge_2.pkl")
ex_4edge_hard_2.to_pickle("Experiments/irrep_DAG_4edge_hard_2.pkl")
ex_4edge_shift_2.to_pickle("Experiments/irrep_DAG_4edge_shift_2.pkl")

### Ex. 3
<img src="Graphics/graph16.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 0], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 9, 10, 11, 14, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 8, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_4edge_3 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_4edge_hard_3 = ex_4edge_3.copy()
ex_4edge_shift_3 = ex_4edge_3.copy()

for row in ex_4edge_3.itertuples():
    row = row.Index
    true_M = ex_4edge_3.iloc[row, 0]

    intervention_index = ex_4edge_3.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_4edge_3.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_4edge_3.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_4edge_shift_3.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_4edge_shift_3.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_4edge_hard_3.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_4edge_hard_3.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_4edge_3 = ex_4edge_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_hard_3 = ex_4edge_hard_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_shift_3 = ex_4edge_shift_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_3.to_pickle("Experiments/irrep_DAG_4edge_3.pkl")
ex_4edge_hard_3.to_pickle("Experiments/irrep_DAG_4edge_hard_3.pkl")
ex_4edge_shift_3.to_pickle("Experiments/irrep_DAG_4edge_shift_3.pkl")

### Ex. 4
<img src="Graphics/graph17.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 8, 9, 10, 12, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 11, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_4edge_4 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_4edge_hard_4 = ex_4edge_4.copy()
ex_4edge_shift_4 = ex_4edge_4.copy()

for row in ex_4edge_4.itertuples():
    row = row.Index
    true_M = ex_4edge_4.iloc[row, 0]

    intervention_index = ex_4edge_4.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_4edge_4.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_4edge_4.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_4edge_shift_4.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_4edge_shift_4.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_4edge_hard_4.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_4edge_hard_4.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_4edge_4 = ex_4edge_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_hard_4 = ex_4edge_hard_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_shift_4 = ex_4edge_shift_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_4.to_pickle("Experiments/irrep_DAG_4edge_4.pkl")
ex_4edge_hard_4.to_pickle("Experiments/irrep_DAG_4edge_hard_4.pkl")
ex_4edge_shift_4.to_pickle("Experiments/irrep_DAG_4edge_shift_4.pkl")

### Ex. 5
<img src="Graphics/graph18.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [0, 3], [1, 0], [1, 2], [1, 3], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 1, 5, 8, 9, 10, 12, 15])
supp_compl = np.array([2, 3, 4, 6, 7, 9, 11, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_4edge_5 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_4edge_hard_5 = ex_4edge_5.copy()
ex_4edge_shift_5 = ex_4edge_5.copy()

for row in ex_4edge_5.itertuples():
    row = row.Index
    true_M = ex_4edge_5.iloc[row, 0]

    intervention_index = ex_4edge_5.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_4edge_5.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_4edge_5.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_4edge_shift_5.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_4edge_shift_5.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_4edge_hard_5.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_4edge_hard_5.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_4edge_5 = ex_4edge_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_hard_5 = ex_4edge_hard_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_shift_5 = ex_4edge_shift_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_5.to_pickle("Experiments/irrep_DAG_4edge_5.pkl")
ex_4edge_hard_5.to_pickle("Experiments/irrep_DAG_4edge_hard_5.pkl")
ex_4edge_shift_5.to_pickle("Experiments/irrep_DAG_4edge_shift_5.pkl")

### Ex. 6
<img src="Graphics/graph19.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [0, 3], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 2]])
support = np.array([0, 1, 5, 8, 10, 11, 12, 15])
supp_compl = np.array([2, 3, 4, 6, 7, 9, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_4edge_6 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_4edge_hard_6 = ex_4edge_6.copy()
ex_4edge_shift_6 = ex_4edge_6.copy()

for row in ex_4edge_6.itertuples():
    row = row.Index
    true_M = ex_4edge_6.iloc[row, 0]

    intervention_index = ex_4edge_6.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_4edge_6.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_4edge_6.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_4edge_shift_6.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_4edge_shift_6.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_4edge_hard_6.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_4edge_hard_6.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_4edge_6 = ex_4edge_6[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_hard_6 = ex_4edge_hard_6[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_shift_6 = ex_4edge_shift_6[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_6.to_pickle("Experiments/irrep_DAG_4edge_6.pkl")
ex_4edge_hard_6.to_pickle("Experiments/irrep_DAG_4edge_hard_6.pkl")
ex_4edge_shift_6.to_pickle("Experiments/irrep_DAG_4edge_shift_6.pkl")

### Ex. 7
<img src="Graphics/graph20.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 2], [1, 0], [1, 2], [1, 3], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 3, 5, 8, 9, 10, 15])
supp_compl = np.array([2, 4, 6, 7, 11, 12, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_4edge_7 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_4edge_hard_7 = ex_4edge_7.copy()
ex_4edge_shift_7 = ex_4edge_7.copy()

for row in ex_4edge_7.itertuples():
    row = row.Index
    true_M = ex_4edge_7.iloc[row, 0]

    intervention_index = ex_4edge_7.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_4edge_7.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_4edge_7.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_4edge_shift_7.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_4edge_shift_7.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_4edge_hard_7.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_4edge_hard_7.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_4edge_7 = ex_4edge_7[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_hard_7 = ex_4edge_hard_7[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_shift_7 = ex_4edge_shift_7[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_7.to_pickle("Experiments/irrep_DAG_4edge_7.pkl")
ex_4edge_hard_7.to_pickle("Experiments/irrep_DAG_4edge_hard_7.pkl")
ex_4edge_shift_7.to_pickle("Experiments/irrep_DAG_4edge_shift_7.pkl")

### Ex. 8
<img src="Graphics/graph21.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 3], [1, 0], [1, 2], [1, 3], [2, 0], [2, 3], [3, 1], [3, 2]])
support = np.array([0, 1, 2, 5, 9, 10, 12, 15])
supp_compl = np.array([3, 4, 6, 7, 8, 11, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_4edge_8 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_4edge_hard_8 = ex_4edge_8.copy()
ex_4edge_shift_8 = ex_4edge_8.copy()

for row in ex_4edge_8.itertuples():
    row = row.Index
    true_M = ex_4edge_8.iloc[row, 0]

    intervention_index = ex_4edge_8.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_4edge_8.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_4edge_8.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_4edge_shift_8.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_4edge_shift_8.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_4edge_hard_8.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_4edge_hard_8.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_4edge_8 = ex_4edge_8[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_hard_8 = ex_4edge_hard_8[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_shift_8 = ex_4edge_shift_8[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_8.to_pickle("Experiments/irrep_DAG_4edge_8.pkl")
ex_4edge_hard_8.to_pickle("Experiments/irrep_DAG_4edge_hard_8.pkl")
ex_4edge_shift_8.to_pickle("Experiments/irrep_DAG_4edge_shift_8.pkl")

### Ex. 9
<img src="Graphics/graph22.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[1, 0], [1, 2], [1, 3], [2, 0], [2, 3], [3, 0], [3, 1], [3, 2]])
support = np.array([0, 1, 2, 3, 5, 9, 10, 15])
supp_compl = np.array([4, 6, 7, 8, 11, 12, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_4edge_9 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_4edge_hard_9 = ex_4edge_9.copy()
ex_4edge_shift_9 = ex_4edge_9.copy()

for row in ex_4edge_9.itertuples():
    row = row.Index
    true_M = ex_4edge_9.iloc[row, 0]

    intervention_index = ex_4edge_9.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_4edge_9.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_4edge_9.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_4edge_shift_9.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_4edge_shift_9.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_4edge_hard_9.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_4edge_hard_9.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_4edge_9 = ex_4edge_9[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_hard_9 = ex_4edge_hard_9[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_shift_9 = ex_4edge_shift_9[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_4edge_9.to_pickle("Experiments/irrep_DAG_4edge_9.pkl")
ex_4edge_hard_9.to_pickle("Experiments/irrep_DAG_4edge_hard_9.pkl")
ex_4edge_shift_9.to_pickle("Experiments/irrep_DAG_4edge_shift_9.pkl")

## 5 nodes

### Ex. 1
<img src="Graphics/graph23.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 3], [2, 1], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 6, 8, 10, 11, 12, 15])
supp_compl = np.array([1, 2, 3, 7, 9, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_5edge_1 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_5edge_hard_1 = ex_5edge_1.copy()
ex_5edge_shift_1 = ex_5edge_1.copy()

for row in ex_5edge_1.itertuples():
    row = row.Index
    true_M = ex_5edge_1.iloc[row, 0]

    intervention_index = ex_5edge_1.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_5edge_1.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_5edge_1.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_5edge_shift_1.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_5edge_shift_1.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_5edge_hard_1.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_5edge_hard_1.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_5edge_1 = ex_5edge_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_hard_1 = ex_5edge_hard_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_shift_1 = ex_5edge_shift_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_1.to_pickle("Experiments/irrep_DAG_5edge_1.pkl")
ex_5edge_hard_1.to_pickle("Experiments/irrep_DAG_5edge_hard_1.pkl")
ex_5edge_shift_1.to_pickle("Experiments/irrep_DAG_5edge_shift_1.pkl")

### Ex. 2
<img src="Graphics/graph24.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 3], [2, 1], [2, 3], [3, 1]])
support = np.array([0, 4, 5, 6, 8, 10, 12, 14, 15])
supp_compl = np.array([1, 2, 3, 7, 9, 11, 13])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_5edge_2 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_5edge_hard_2 = ex_5edge_2.copy()
ex_5edge_shift_2 = ex_5edge_2.copy()

for row in ex_5edge_2.itertuples():
    row = row.Index
    true_M = ex_5edge_2.iloc[row, 0]

    intervention_index = ex_5edge_2.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_5edge_2.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_5edge_2.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_5edge_shift_2.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_5edge_shift_2.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_5edge_hard_2.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_5edge_hard_2.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_5edge_2 = ex_5edge_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_hard_2 = ex_5edge_hard_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_shift_2 = ex_5edge_shift_2[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_2.to_pickle("Experiments/irrep_DAG_5edge_2.pkl")
ex_5edge_hard_2.to_pickle("Experiments/irrep_DAG_5edge_hard_2.pkl")
ex_5edge_shift_2.to_pickle("Experiments/irrep_DAG_5edge_shift_2.pkl")

### Ex. 3
<img src="Graphics/graph25.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 8, 9, 10, 11, 12, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_5edge_3 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_5edge_hard_3 = ex_5edge_3.copy()
ex_5edge_shift_3 = ex_5edge_3.copy()

for row in ex_5edge_3.itertuples():
    row = row.Index
    true_M = ex_5edge_3.iloc[row, 0]

    intervention_index = ex_5edge_3.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_5edge_3.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_5edge_3.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_5edge_shift_3.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_5edge_shift_3.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_5edge_hard_3.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_5edge_hard_3.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_5edge_3 = ex_5edge_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_hard_3 = ex_5edge_hard_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_shift_3 = ex_5edge_shift_3[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_3.to_pickle("Experiments/irrep_DAG_5edge_3.pkl")
ex_5edge_hard_3.to_pickle("Experiments/irrep_DAG_5edge_hard_3.pkl")
ex_5edge_shift_3.to_pickle("Experiments/irrep_DAG_5edge_shift_3.pkl")

### Ex. 4
<img src="Graphics/graph26.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [2, 0], [3, 1], [3, 2]])
support = np.array([0, 4, 5, 7, 9, 10, 11, 12, 15])
supp_compl = np.array([1, 2, 3, 6, 8, 13, 14])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_5edge_4 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_5edge_hard_4 = ex_5edge_4.copy()
ex_5edge_shift_4 = ex_5edge_4.copy()

for row in ex_5edge_4.itertuples():
    row = row.Index
    true_M = ex_5edge_4.iloc[row, 0]

    intervention_index = ex_5edge_4.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_5edge_4.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_5edge_4.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_5edge_shift_4.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_5edge_shift_4.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_5edge_hard_4.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_5edge_hard_4.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_5edge_4 = ex_5edge_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_hard_4 = ex_5edge_hard_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_shift_4 = ex_5edge_shift_4[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_4.to_pickle("Experiments/irrep_DAG_5edge_4.pkl")
ex_5edge_hard_4.to_pickle("Experiments/irrep_DAG_5edge_hard_4.pkl")
ex_5edge_shift_4.to_pickle("Experiments/irrep_DAG_5edge_shift_4.pkl")

### Ex. 5
<img src="Graphics/graph27.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 0], [2, 3]])
support = np.array([0, 4, 5, 9, 10, 12, 13, 14, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 8, 11])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_5edge_5 = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_5edge_hard_5 = ex_5edge_5.copy()
ex_5edge_shift_5 = ex_5edge_5.copy()

for row in ex_5edge_5.itertuples():
    row = row.Index
    true_M = ex_5edge_5.iloc[row, 0]

    intervention_index = ex_5edge_5.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_5edge_5.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_5edge_5.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_5edge_shift_5.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_5edge_shift_5.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_5edge_hard_5.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_5edge_hard_5.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_5edge_5 = ex_5edge_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_hard_5 = ex_5edge_hard_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_shift_5 = ex_5edge_shift_5[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_5edge_5.to_pickle("Experiments/irrep_DAG_5edge_5.pkl")
ex_5edge_hard_5.to_pickle("Experiments/irrep_DAG_5edge_hard_5.pkl")
ex_5edge_shift_5.to_pickle("Experiments/irrep_DAG_5edge_shift_5.pkl")

## 6 nodes
<img src="Graphics/graph28.png" alt="Drawing" style="width: 100px;"/>

In [ ]:
p = 4
seed = 0
index = 0

index_zero = np.array([[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]])
support = np.array([0, 4, 5, 8, 9, 10, 12, 13, 14, 15])
supp_compl = np.array([1, 2, 3, 6, 7, 11])

normal_C = volatility_param * np.eye(p)
C_hard = [np.delete(np.delete(normal_C, i, axis=0), i, axis=1) for i in range(p)]

temp1 = pd.DataFrame(np.empty(shape=(num_exp, 3), dtype=object), columns=["True M", "Irrep. fulfilled", "Weak irrep. fulfilled"])

while index < num_exp:
    np.random.seed(seed=seed)
    M = np.random.uniform(low=-1.0, high=1.0, size=(p, p))
    M[index_zero[:, 0], index_zero[:, 1]] = 0
    seed += 1
    eigenvalues = np.real(np.linalg.eig(M).eigenvalues)
    if np.any(eigenvalues >= 0):
        continue
    temp1.iloc[index, 0] = M
    index += 1

ex_6edge = pd.concat([temp1.assign(Intervention = i) for i in range(p)], ignore_index=True)
ex_6edge_hard_1 = ex_6edge.copy()
ex_6edge_shift_1 = ex_6edge.copy()

for row in ex_6edge.itertuples():
    row = row.Index
    true_M = ex_6edge.iloc[row, 0]

    intervention_index = ex_6edge.iloc[row, 3]
    unit_vector = np.eye(p)[:, intervention_index]

    M_hard = np.delete(np.delete(true_M, intervention_index, axis=0), intervention_index, axis=1)
    m_hard = np.delete(true_M[:, intervention_index], intervention_index, axis=0)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-normal_C)
    true_cov_hard = solve_continuous_lyapunov(a=M_hard, q=-C_hard[intervention_index])

    true_mean_shift = (shift_param * np.linalg.inv(a=true_M)).dot(unit_vector)
    true_mean_hard = (- hard_param * np.linalg.inv(a=M_hard)).dot(m_hard)

    A_Sigma = create_A_Sigma(cov=true_cov, p=p)
    A_Sigma_shift = create_A_shift(cov=true_cov, mean_int=true_mean_shift, p=p)
    A_Sigma_hard = create_A_hard(cov=true_cov, cov_int=true_cov_hard, mean_int=true_mean_hard, p=p, index=intervention_index)

    irrep_cond_no = irrep_cond(A=A_Sigma, support=support, supp_compl=supp_compl)
    weak_irrep_cond_no = weak_irrep_cond(M=true_M, A=A_Sigma, support=support, supp_compl=supp_compl)
    irrep_cond_shift = irrep_cond(A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    weak_irrep_cond_shift = weak_irrep_cond(M=true_M, A=A_Sigma_shift, support=support, supp_compl=supp_compl)
    irrep_cond_hard = irrep_cond(A=A_Sigma_hard, support=support, supp_compl=supp_compl)
    weak_irrep_cond_hard = weak_irrep_cond(M=true_M, A=A_Sigma_hard, support=support, supp_compl=supp_compl)

    ex_6edge.iloc[row, 1] = (0 <= irrep_cond_no < 1)
    ex_6edge.iloc[row, 2] = (0 <= weak_irrep_cond_no <= 1)
    ex_6edge_shift_1.iloc[row, 1] = (0 <= irrep_cond_shift < 1)
    ex_6edge_shift_1.iloc[row, 2] = (0 <= weak_irrep_cond_shift <= 1)
    ex_6edge_hard_1.iloc[row, 1] = (0 <= irrep_cond_hard < 1)
    ex_6edge_hard_1.iloc[row, 2] = (0 <= weak_irrep_cond_hard <= 1)


ex_6edge = ex_6edge[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_6edge_hard_1 = ex_6edge_hard_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_6edge_shift_1 = ex_6edge_shift_1[["Intervention", "True M", "Irrep. fulfilled", "Weak irrep. fulfilled"]]
ex_6edge.to_pickle("Experiments/irrep_DAG_6edge_1.pkl")
ex_6edge_hard_1.to_pickle("Experiments/irrep_DAG_6edge_hard_1.pkl")
ex_6edge_shift_1.to_pickle("Experiments/irrep_DAG_6edge_shift_1.pkl")